<a href="https://colab.research.google.com/github/mgoncerz/ml_zaliczenie_projekt/blob/master/projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The idea is to compare the accuracy of various ML methods in reckognizing handwritten letters from alphabets of different complexity.

Loading libraries:

In [0]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.font_manager import FontProperties

import pandas as pd

import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, adam
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.utils.vis_utils import plot_model
from IPython.display import Image 

from sklearn.decomposition import PCA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn import metrics

Hiragana:

1. Loading data (https://www.kaggle.com/anokas/kuzushiji):

In [0]:
images_train = np.load('kmnist-train-imgs.npz')['arr_0']
images_test = np.load('kmnist-test-imgs.npz')['arr_0']

data_length = images_train[0].shape[0] * images_train[0].shape[1]

x_train = images_train.reshape(len(images_train), data_length).astype('float32')
x_test = images_test.reshape(len(images_test), data_length).astype('float32')

labels_train = np.load('kmnist-train-labels.npz')['arr_0']
labels_test = np.load('kmnist-test-labels.npz')['arr_0']

y_train = keras.utils.to_categorical(labels_train, 10)
y_test = keras.utils.to_categorical(labels_test, 10)

labels_map = pd.read_csv("kmnist_classmap.csv", encoding="utf-8")

2. Input data analysis:

In [0]:
print(labels_map)

fig, axes = plt.subplots(figsize=(20,20))

for i in range(100):
  ax = fig.add_subplot(10,10,i+1,xticks=[], yticks=[])
  ax.imshow(images_train[i], cmap=plt.cm.binary, interpolation='nearest')
  ax.text(1, 4, str(labels_train[i]))

In [0]:
projection = PCA(n_components=2).fit_transform(x_train)
plt.scatter(projection[:, 0], projection[:, 1], c=labels_train.astype(int), cmap=plt.get_cmap('Paired', 10), vmin=-0.5, vmax=9.5)
plt.colorbar()

3. Simple approach (Fisher):



In [0]:
print("Fisher:")
f_model = LinearDiscriminantAnalysis()
f_model.fit(x_train, labels_train) #note this takes labels as single numbers per input element, not transformed to dij form
f_prediction = f_model.predict(x_test)

print("Score: ", f_model.score(x_test, labels_test))

fig, axes = plt.subplots(figsize=(20,20))

print(metrics.classification_report(labels_test, f_prediction))

f_cfm = metrics.plot_confusion_matrix(f_model, x_test, labels_test, cmap=plt.cm.Blues, normalize = "true", ax = axes)
f_cfm.ax_.set_title("Fisher confusion matrix")


4. k-neighbours

In [0]:
print("k-neighbors:")
kneighbors_model = KNeighborsClassifier(3)
kneighbors_model.fit(x_train[], labels_train[]) #note this takes labels as single numbers per input element, not transformed to dij form
kneighbors_prediction = kneighbors_model.predict(x_test)

print("Score: ", kneighbors_model.score(x_test, labels_test))

fig, axes = plt.subplots(figsize=(20,20))

print(metrics.classification_report(labels_test, kneighbors_prediction))

f_cfm = metrics.plot_confusion_matrix(kneighbors_model, x_test, labels_test, cmap=plt.cm.Blues, normalize = "true", ax = axes)
f_cfm.ax_.set_title("k-neighbors confusion matrix")

5. Boosted Decision Trees (AdaBoost)

In [0]:
print("AdaBoost:")

adaboost_model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=60),n_estimators=10)

adaboost_model.fit(x_train, labels_train) #note this takes labels as single numbers per input element, not transformed to dij form
adaboost_prediction = adaboost_model.predict(x_test)

print("Score: ", adaboost_model.score(x_test, labels_test))

fig, axes = plt.subplots(figsize=(20,20))

print(metrics.classification_report(labels_test, adaboost_prediction))

adaboost_cfm = metrics.plot_confusion_matrix(adaboost_model, x_test, labels_test, cmap=plt.cm.Blues, normalize = "true", ax = axes)
adaboost_cfm.ax_.set_title("AdaBoost confusion matrix")


6. Simple Neural Network